<a href="https://colab.research.google.com/github/PuchToTalk/ConsumerEdgeHackaton/blob/data/Data_extraction_wiki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install wikipedia-api


  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia-api: filename=Wikipedia_API-0.7.1-py3-none-any.whl size=14346 sha256=cad3561701e03e22a6c2e966ffec2ed1c75e4fa1a524ccf948f11e605b73575c
  Stored in directory: /root/.cache/pip/wheels/4c/96/18/b9201cc3e8b47b02b510460210cfd832ccf10c0c4dd0522962
Successfully built wikipedia-api


# Exploration Draft

## Test Simple Extraction from Wikipedia


In [ ]:
import requests
import csv
from bs4 import BeautifulSoup

def get_wikipedia_data_csv(title, language="en"):
    # URL de l'API Wikipedia
    url = f"https://{language}.wikipedia.org/w/api.php"

    # Paramètres pour extraire les différentes informations de la page
    params = {
        "action": "query",
        "format": "json",
        "prop": "extracts|links|categories|images|info",
        "exintro": True,
        "titles": title,
        "pllimit": "max",
        "cllimit": "max",
        "inprop": "url"
    }

    # Requête à l'API Wikipedia
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        page_id = next(iter(data['query']['pages']))
        if page_id == "-1":
            print(f"La page '{title}' n'existe pas.")
        else:
            page = data['query']['pages'][page_id]

            # Extraction des données de l'article
            page_title = page.get('title', 'N/A')
            page_url = page.get('fullurl', 'N/A')

            # Extraire et nettoyer le texte de l'article
            raw_extract = page.get('extract', 'N/A')
            if raw_extract != 'N/A':
                soup = BeautifulSoup(raw_extract, "html.parser")
                article_extract = soup.get_text(strip=True)  # Supprime toutes les balises HTML
            else:
                article_extract = 'N/A'

            # Calcul du nombre de mots dans l'extrait nettoyé
            word_count = len(article_extract.split()) if article_extract != 'N/A' else 0

            # Extraction des liens internes
            internal_links = "; ".join([link['title'] for link in page.get('links', []) if 'title' in link])

            # Extraction des catégories
            categories = "; ".join([category['title'] for category in page.get('categories', []) if 'title' in category])

            # Extraction des images (nom de fichier seulement)
            images = "; ".join([image['title'] for image in page.get('images', []) if 'title' in image])

            # Écriture dans le fichier CSV
            with open('wikipedia_data.csv', mode='a', newline='', encoding='utf-8') as file:
                writer = csv.writer(file)
                writer.writerow([page_title, page_url, article_extract, word_count, internal_links, categories, images])
    else:
        print("Erreur lors de la récupération des données.")

# Initialiser le fichier CSV avec les en-têtes
with open('wikipedia_data.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["page_title", "page_url", "article_extract", "word_count", "internal_links", "categories", "images"])

# Appeler la fonction pour obtenir les détails de la page "Christmas" et les enregistrer dans un fichier CSV
get_wikipedia_data_csv("Christmas")
get_wikipedia_data_csv("Artificial intelligence")


# I - Wikipedia Scraping & Generation of the dataset

In [ ]:
import requests
import csv

def verify_wikipedia_pages(titles, language="fr"):
    # URL de base de l'API Wikipedia
    base_url = f"https://{language}.wikipedia.org/w/api.php"

    # Initialiser le fichier CSV avec les résultats
    with open('wikipedia_pages_verification.csv', mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["title", "exists", "wikipedia_url"])

        # Vérifier chaque titre
        for title in titles:
            params = {
                "action": "query",
                "format": "json",
                "titles": title
            }

            # Requête à l'API Wikipedia
            response = requests.get(base_url, params=params)

            if response.status_code == 200:
                data = response.json()
                page_id = next(iter(data['query']['pages']))

                if page_id == "-1":
                    # La page n'existe pas
                    writer.writerow([title, False, "N/A"])
                else:
                    # La page existe
                    page_url = f"https://{language}.wikipedia.org/wiki/{title.replace(' ', '_')}"
                    writer.writerow([title, True, page_url])
            else:
                print(f"Erreur lors de la vérification du titre '{title}'.")

# Liste des titres à vérifier
titles = [
    "Alice au Pays des Merveilles", "Peter Pan", "Robin des Bois", "Mulan", "Pocahontas",
    "Le Livre de la Jungle", "Bambi", "Le Roi Lion", "Les Aventures de Winnie l’Ourson",
    "La Princesse et la Grenouille", "La Reine des Neiges", "Moana (Vaiana)", "Blanche-Neige et Rose-Rouge",
    "Ali Baba et les Quarante Voleurs", "Sinbad le Marin", "Le Petit Poucet", "Peau d’Âne",
    "Les Aventures de Tintin", "Astérix et Obélix", "Le Magicien d’Oz", "Oliver Twist",
    "Les Douze Princesses Dansantes", "Le Joueur de Flûte de Hamelin", "La Princesse au Petit Pois",
    "Les Musiciens de Brême", "Ali et le Génie", "Anastasia", "La Petite Fille aux Allumettes",
    "Les Cygnes Sauvages", "Madame Bovary (version simplifiée)", "Heidi", "L’Étrange Noël de Monsieur Jack",
    "Wallace et Gromit", "Chicken Run", "Madagascar", "Kung Fu Panda", "Toy Story",
    "Le Monde de Nemo", "Les Indestructibles", "Shrek", "Cars", "Ratatouille", "Monstres et Cie",
    "Coco", "Les Aventures de Tom Sawyer", "Les Quatre Filles du Docteur March", "Mary Poppins",
    "La Mélodie du Bonheur", "Pippi Longstocking (Fifi Brindacier)", "Peter et Elliott le Dragon",
    "Ferdinand", "Le Petit Prince", "Taram et le Chaudron Magique", "Balto", "Spirit, l’Étalon des Plaines",
    "Les Aristochats", "Le Renard et l’Enfant", "Les Animaux du Bois de Quat’sous", "Bernard et Bianca",
    "Tinker Bell", "Frère des Ours", "Lilo et Stitch", "Rio", "Brave (Rebelle)", "Horton Hears a Who!",
    "Les Schtroumpfs", "Zootopia", "Les Trolls", "La Garde du Roi Lion", "La Légende de Korra",
    "Le Petit Nicolas", "Alice de l’Autre Côté du Miroir", "Charlie et la Chocolaterie", "Le Géant de Fer",
    "Mia et le Migou", "Les Enfants Loups, Ame et Yuki", "Ponyo sur la falaise", "Mon voisin Totoro",
    "Kiki la petite sorcière", "Le Voyage de Chihiro", "Princesse Mononoké", "Le Château ambulant",
    "Nausicaä de la vallée du vent", "Les Enfants de la mer", "Le Château dans le ciel", "Coraline",
    "Anastasia", "Ernest et Célestine", "Paddington", "Le Gruffalo", "Max et les Maximonstres",
    "Le Roi et l’Oiseau", "Kirikou et la Sorcière", "Azur et Asmar", "L’Histoire sans Fin",
    "La Fabuleuse Gilly Hopkins", "Un Chant de Noël", "Le Secret de la Petite Sirène", "Les Trois Brigands",
    "Le Renard et l’Érable"
]

# Vérifier chaque page Wikipedia
verify_wikipedia_pages(titles)


In [ ]:
import os

# Get the current working directory
current_path = os.getcwd()
print(current_path)


/content


## Fixing missing values

In [ ]:
import csv

# URLs fournis à ajouter
additional_urls = {
    "Les Aventures de Winnie l’Ourson": "https://fr.wikipedia.org/wiki/Winnie_l%27ourson",
    "La Reine des Neiges": "https://fr.wikipedia.org/wiki/La_Reine_des_neiges",
    "Les Douze Princesses Dansantes": "https://fr.wikipedia.org/wiki/Le_Bal_des_douze_princesses",
    "Le Joueur de Flûte de Hamelin": "https://fr.wikipedia.org/wiki/Le_Joueur_de_fl%C3%BBte_de_Hamelin",
    "La Princesse au Petit Pois": "https://fr.wikipedia.org/wiki/La_Princesse_au_petit_pois",
    "Ali et le Génie": "https://fr.wikipedia.org/wiki/Aladdin_(Disney)",
    "La Petite Fille aux Allumettes": "https://fr.wikipedia.org/wiki/La_Petite_Fille_aux_allumettes",
    "Les Cygnes Sauvages": "https://fr.wikipedia.org/wiki/Les_Cygnes_sauvages_(Andersen)",
    "Madame Bovary (version simplifiée)": "https://fr.wikipedia.org/wiki/Madame_Bovary",
    "L’Étrange Noël de Monsieur Jack": "https://fr.wikipedia.org/wiki/L%27%C3%89trange_No%C3%ABl_de_monsieur_Jack",
    "La Mélodie du Bonheur": "https://fr.wikipedia.org/wiki/La_M%C3%A9lodie_du_bonheur_(film,_1965)",
    "Pippi Longstocking (Fifi Brindacier)": "https://fr.wikipedia.org/wiki/Fifi_Brindacier",
    "Taram et le Chaudron Magique": "https://fr.wikipedia.org/wiki/Taram_et_le_Chaudron_magique",
    "Spirit, l’Étalon des Plaines": "https://fr.wikipedia.org/wiki/Spirit,_l%27%C3%A9talon_des_plaines",
    "Le Renard et l’Enfant": "https://fr.wikipedia.org/wiki/Le_Renard_et_l%27Enfant",
    "Brave (Rebelle)": "https://fr.wikipedia.org/wiki/Rebelle_(film_d%27animation,_2012)",
    "Alice de l’Autre Côté du Miroir": "https://fr.wikipedia.org/wiki/Alice_de_l%27autre_c%C3%B4t%C3%A9_du_miroir",
    "Le Géant de Fer": "https://fr.wikipedia.org/wiki/Le_G%C3%A9ant_de_fer",
    "Les Enfants Loups, Ame et Yuki": "https://fr.wikipedia.org/wiki/Les_Enfants_loups,_Ame_et_Yuki",
    "Le Roi et l’Oiseau": "https://fr.wikipedia.org/wiki/Le_Roi_et_l%27Oiseau",
    "L’Histoire sans Fin": "https://fr.wikipedia.org/wiki/L%27Histoire_sans_fin_(film)",
    "Le Renard et l’Érable": "https://fr.wikipedia.org/wiki/Le_Renard_et_les_Raisins",
    "Moana (Vaiana)" : "https://fr.wikipedia.org/wiki/Vaiana_:_La_L%C3%A9gende_du_bout_du_monde"
    "Coco" : "https://fr.wikipedia.org/wiki/Coco_(film,_2017)"
}

# Chemin du fichier CSV téléchargé
input_csv_path = '/content/wikipedia_pages_verification.csv'
output_csv_path = '/content/wikipedia_pages_verification_updated_final.csv'

# Mise à jour du fichier CSV avec les URLs supplémentaires
with open(input_csv_path, mode='r', encoding='utf-8') as infile, open(output_csv_path, mode='w', newline='', encoding='utf-8') as outfile:
    reader = csv.DictReader(infile)
    fieldnames = reader.fieldnames
    writer = csv.DictWriter(outfile, fieldnames=fieldnames)

    # Écrire les en-têtes dans le fichier de sortie
    writer.writeheader()

    # Mise à jour des lignes existantes avec les URLs supplémentaires si nécessaires
    for row in reader:
        title = row['title']
        if title in additional_urls and (row['wikipedia_url'] == "N/A" or not row['wikipedia_url']):
            row['exists'] = True
            row['wikipedia_url'] = additional_urls[title]
        writer.writerow(row)

# Chemin du fichier CSV mis à jour
output_csv_path


'/content/wikipedia_pages_verification_updated_final.csv'

## Generate the corpus & Merge results

In [ ]:
import requests
import csv
from bs4 import BeautifulSoup

# Fonction pour obtenir les détails de Wikipédia
def get_wikipedia_data_csv(title, language="fr"):
    # URL de l'API Wikipedia
    url = f"https://{language}.wikipedia.org/w/api.php"

    # Paramètres pour extraire les différentes informations de la page
    params = {
        "action": "query",
        "format": "json",
        "prop": "extracts|links|categories|images|info",
        "exintro": True,
        "titles": title,
        "pllimit": "max",
        "cllimit": "max",
        "inprop": "url"
    }

    # Requête à l'API Wikipedia
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        page_id = next(iter(data['query']['pages']))
        if page_id == "-1":
            print(f"La page '{title}' n'existe pas.")
        else:
            page = data['query']['pages'][page_id]

            # Extraction des données de l'article
            page_title = page.get('title', 'N/A')
            page_url = page.get('fullurl', 'N/A')

            # Extraire et nettoyer le texte de l'article
            raw_extract = page.get('extract', 'N/A')
            if raw_extract != 'N/A':
                soup = BeautifulSoup(raw_extract, "html.parser")
                article_extract = soup.get_text(strip=True)  # Supprime toutes les balises HTML
            else:
                article_extract = 'N/A'

            # Calcul du nombre de mots dans l'extrait nettoyé
            word_count = len(article_extract.split()) if article_extract != 'N/A' else 0

            # Extraction des liens internes
            internal_links = "; ".join([link['title'] for link in page.get('links', []) if 'title' in link])

            # Extraction des catégories
            categories = "; ".join([category['title'] for category in page.get('categories', []) if 'title' in category])

            # Extraction des images (nom de fichier seulement)
            images = "; ".join([image['title'] for image in page.get('images', []) if 'title' in image])

            # Retourner les données sous forme de dictionnaire
            return {
                "page_title": page_title,
                "page_url": page_url,
                "article_extract": article_extract,
                "word_count": word_count,
                "internal_links": internal_links,
                "categories": categories,
                "images": images
            }
    else:
        print("Erreur lors de la récupération des données.")
        return None

# Chemin du fichier CSV téléchargé
input_csv_path = '/content/wikipedia_pages_verification_updated_final.csv'
output_csv_path = '/content/wikipedia_data_enriched.csv'

# Lire le fichier CSV et appliquer la fonction à chaque titre
with open(input_csv_path, mode='r', encoding='utf-8') as infile, open(output_csv_path, mode='w', newline='', encoding='utf-8') as outfile:
    reader = csv.DictReader(infile)
    fieldnames = ["page_title", "page_url", "article_extract", "word_count", "internal_links", "categories", "images"]
    writer = csv.DictWriter(outfile, fieldnames=fieldnames)

    # Écrire les en-têtes dans le fichier de sortie
    writer.writeheader()

    # Appliquer la fonction à chaque ligne
    for row in reader:
        title = row['title']
        if row['exists'].lower() == 'true':
            wikipedia_data = get_wikipedia_data_csv(title)
            if wikipedia_data:
                writer.writerow(wikipedia_data)

# Chemin du fichier CSV enrichi
output_csv_path


La page 'Les Aventures de Winnie l’Ourson' n'existe pas.
La page 'La Reine des Neiges' n'existe pas.
La page 'Moana (Vaiana)' n'existe pas.
La page 'Les Douze Princesses Dansantes' n'existe pas.
La page 'Le Joueur de Flûte de Hamelin' n'existe pas.
La page 'La Princesse au Petit Pois' n'existe pas.
La page 'Ali et le Génie' n'existe pas.
La page 'La Petite Fille aux Allumettes' n'existe pas.
La page 'Les Cygnes Sauvages' n'existe pas.
La page 'Madame Bovary (version simplifiée)' n'existe pas.
La page 'L’Étrange Noël de Monsieur Jack' n'existe pas.
La page 'La Mélodie du Bonheur' n'existe pas.
La page 'Pippi Longstocking (Fifi Brindacier)' n'existe pas.
La page 'Taram et le Chaudron Magique' n'existe pas.
La page 'Spirit, l’Étalon des Plaines' n'existe pas.
La page 'Le Renard et l’Enfant' n'existe pas.
La page 'Brave (Rebelle)' n'existe pas.
La page 'Alice de l’Autre Côté du Miroir' n'existe pas.
La page 'Le Géant de Fer' n'existe pas.
La page 'Les Enfants Loups, Ame et Yuki' n'existe p

'/content/wikipedia_data_enriched.csv'

In [ ]:
import csv

# URLs fournis à ajouter
replacement_urls = {
    "Les Aventures de Winnie l’Ourson": "https://fr.wikipedia.org/wiki/Les_Aventures_d%27Alice_au_pays_des_merveilles",
    "La Reine des Neiges": "https://fr.wikipedia.org/wiki/Le_Livre_de_la_jungle",
    "Moana (Vaiana)": "https://fr.wikipedia.org/wiki/Le_Roi_lion_(film,_1994)",
    "Les Douze Princesses Dansantes": "https://fr.wikipedia.org/wiki/Peau_d%27Âne",
    "Le Joueur de Flûte de Hamelin": "https://fr.wikipedia.org/wiki/Astérix",
    "La Princesse au Petit Pois": "https://fr.wikipedia.org/wiki/Le_Magicien_d%27Oz",
    "Ali et le Génie": "https://fr.wikipedia.org/wiki/Shrek_(film)",
    "La Petite Fille aux Allumettes": "https://fr.wikipedia.org/wiki/Coco_(film,_2017)",
    "Les Cygnes Sauvages": "https://fr.wikipedia.org/wiki/Les_Quatre_Filles_du_docteur_March",
    "Madame Bovary (version simplifiée)": "https://fr.wikipedia.org/wiki/Peter_et_Elliott_le_dragon",
    "L’Étrange Noël de Monsieur Jack": "https://fr.wikipedia.org/wiki/Les_Animaux_du_Bois_de_Quat%27sous",
    "La Mélodie du Bonheur": "https://fr.wikipedia.org/wiki/Les_Aventures_de_Bernard_et_Bianca",
    "Pippi Longstocking (Fifi Brindacier)": "https://fr.wikipedia.org/wiki/Fée_Clochette",
    "Taram et le Chaudron Magique": "https://fr.wikipedia.org/wiki/Frère_des_ours",
    "Spirit, l’Étalon des Plaines": "https://fr.wikipedia.org/wiki/Horton_(film)",
    "Le Renard et l’Enfant": "https://fr.wikipedia.org/wiki/Zootopie",
    "Brave (Rebelle)": "https://fr.wikipedia.org/wiki/La_Garde_du_Roi_lion",
    "Alice de l’Autre Côté du Miroir": "https://fr.wikipedia.org/wiki/Nausicaä_de_la_Vallée_du_Vent_(film_d%27animation)",
    "Le Géant de Fer": "https://fr.wikipedia.org/wiki/Un_chant_de_Noël",
    "Les Trolls": "https://fr.wikipedia.org/wiki/Les_Trolls_(film)",
    "Les Enfants de la mer": "https://fr.wikipedia.org/wiki/Les_Enfants_de_la_mer_(film)",
    "Nausicaä de la Vallée du Vent": "https://fr.wikipedia.org/wiki/Nausicaä_de_la_Vallée_du_Vent_(film_d%27animation)",
    "Ratatouille":"https://fr.wikipedia.org/wiki/Ratatouille_(film)",
    "Cars": "https://fr.wikipedia.org/wiki/Cars_(film)",
    "Shrek": "https://fr.wikipedia.org/wiki/Shrek_(film)",
    "Coco": "https://fr.wikipedia.org/wiki/Coco_(film,_2017)"
}

# Chemin du fichier CSV téléchargé
input_csv_path = '/content/final-wiki.csv'
output_csv_path = '/content/wiki1.csv'

# Mise à jour du fichier CSV avec les URLs de remplacement
with open(input_csv_path, mode='r', encoding='utf-8') as infile, open(output_csv_path, mode='w', newline='', encoding='utf-8') as outfile:
    reader = csv.DictReader(infile)
    fieldnames = reader.fieldnames
    writer = csv.DictWriter(outfile, fieldnames=fieldnames)

    # Écrire les en-têtes dans le fichier de sortie
    writer.writeheader()

    # Mise à jour des lignes existantes avec les URLs de remplacement si nécessaire
    for row in reader:
        title = row['title']
        if title in replacement_urls:
            row['exists'] = True
            row['wikipedia_url'] = replacement_urls[title]
        writer.writerow(row)

# Chemin du fichier CSV mis à jour
output_csv_path

'/content/wiki1.csv'

In [ ]:
import csv
import sys

# Augmenter la limite maximale de taille de champ
csv.field_size_limit(sys.maxsize)

# URLs fournies à ajouter ou mettre à jour
new_urls = {
    "Les Aventures de Winnie l’Ourson": "https://fr.wikipedia.org/wiki/Winnie_l%27ourson",
    "La Reine des Neiges": "https://fr.wikipedia.org/wiki/La_Reine_des_neiges",
    "Les Douze Princesses Dansantes": "https://fr.wikipedia.org/wiki/Le_Bal_des_douze_princesses",
    "Le Joueur de Flûte de Hamelin": "https://fr.wikipedia.org/wiki/Le_Joueur_de_fl%C3%BBte_de_Hamelin",
    "La Princesse au Petit Pois": "https://fr.wikipedia.org/wiki/La_Princesse_au_petit_pois",
    "Ali et le Génie": "https://fr.wikipedia.org/wiki/Aladdin_(Disney)",
    "La Petite Fille aux Allumettes": "https://fr.wikipedia.org/wiki/La_Petite_Fille_aux_allumettes",
    "Les Cygnes Sauvages": "https://fr.wikipedia.org/wiki/Les_Cygnes_sauvages_(Andersen)",
    "Madame Bovary (version simplifiée)": "https://fr.wikipedia.org/wiki/Madame_Bovary",
    "L’Étrange Noël de Monsieur Jack": "https://fr.wikipedia.org/wiki/L%27%C3%89trange_No%C3%ABl_de_monsieur_Jack",
    "La Mélodie du Bonheur": "https://fr.wikipedia.org/wiki/La_M%C3%A9lodie_du_bonheur_(film,_1965)",
    "Pippi Longstocking (Fifi Brindacier)": "https://fr.wikipedia.org/wiki/Fifi_Brindacier",
    "Taram et le Chaudron Magique": "https://fr.wikipedia.org/wiki/Taram_et_le_Chaudron_magique",
    "Spirit, l’Étalon des Plaines": "https://fr.wikipedia.org/wiki/Spirit,_l%27%C3%A9talon_des_plaines",
    "Le Renard et l’Enfant": "https://fr.wikipedia.org/wiki/Le_Renard_et_l%27Enfant",
    "Brave (Rebelle)": "https://fr.wikipedia.org/wiki/Rebelle_(film_d%27animation,_2012)",
    "Alice de l’Autre Côté du Miroir": "https://fr.wikipedia.org/wiki/Alice_de_l%27autre_c%C3%B4t%C3%A9_du_miroir",
    "Le Géant de Fer": "https://fr.wikipedia.org/wiki/Le_G%C3%A9ant_de_fer",
    "Les Enfants Loups, Ame et Yuki": "https://fr.wikipedia.org/wiki/Les_Enfants_loups,_Ame_et_Yuki",
    "Le Roi et l’Oiseau": "https://fr.wikipedia.org/wiki/Le_Roi_et_l%27Oiseau",
    "L’Histoire sans Fin": "https://fr.wikipedia.org/wiki/L%27Histoire_sans_fin_(film)",
    "Le Renard et l’Érable": "https://fr.wikipedia.org/wiki/Le_Renard_et_les_Raisins",
    "Moana (Vaiana)": "https://fr.wikipedia.org/wiki/Vaiana_:_La_L%C3%A9gende_du_bout_du_monde",
    "Coco": "https://fr.wikipedia.org/wiki/Coco_(film,_2017)",
    "Les Trolls": "https://fr.wikipedia.org/wiki/Les_Trolls_(film)",
    "Les Enfants de la mer": "https://fr.wikipedia.org/wiki/Les_Enfants_de_la_mer_(film)",
    "Nausicaä de la Vallée du Vent": "https://fr.wikipedia.org/wiki/Nausicaä_de_la_Vallée_du_Vent_(film_d%27animation)",
    "Ratatouille": "https://fr.wikipedia.org/wiki/Ratatouille_(film)",
    "Cars": "https://fr.wikipedia.org/wiki/Cars_(film)",
    "Shrek": "https://fr.wikipedia.org/wiki/Shrek_(film)"
}

# URLs supplémentaires à mettre à jour
additional_urls = {
    "Bambi": "https://fr.wikipedia.org/wiki/Bambi_(film,_1942)",
    "Le Livre de la jungle": "https://fr.wikipedia.org/wiki/Le_Livre_de_la_jungle_(film,_1967)",
    "Anastasia": "https://fr.wikipedia.org/wiki/Anastasia_(film,_1997)",
    "Heidi": "https://fr.wikipedia.org/wiki/Heidi_(série_télévisée_d%27animation)",
    "Mary Poppins": "https://fr.wikipedia.org/wiki/Mary_Poppins_(film)",
    "Les Quatre Filles du docteur March (homonymie)": "https://fr.wikipedia.org/wiki/Les_Quatre_Filles_du_docteur_March",
    "Les Trois Brigands": "https://fr.wikipedia.org/wiki/Les_Trois_Brigands_(film)"
}

# Fusionner toutes les nouvelles URLs
all_urls = {**new_urls, **additional_urls}

# Chemin du fichier CSV téléchargé
input_csv_path = '/mnt/data/first-result.csv'
output_csv_path = '/mnt/data/first-result-overwritten-final-complete.csv'

# Mise à jour du fichier CSV avec les nouvelles URLs en utilisant la colonne 'page_title' pour remplacer la colonne 'page_url'
with open(input_csv_path, mode='r', encoding='utf-8') as infile, open(output_csv_path, mode='w', newline='', encoding='utf-8') as outfile:
    reader = csv.DictReader(infile)
    fieldnames = reader.fieldnames
    writer = csv.DictWriter(outfile, fieldnames=fieldnames)

    # Écrire les en-têtes dans le fichier de sortie
    writer.writeheader()

    # Mise à jour des lignes existantes avec les nouvelles URLs pour 'page_url'
    for row in reader:
        title = row['page_title']
        if title in all_urls:
            row['page_url'] = all_urls[title]
        writer.writerow(row)

# Chemin du fichier CSV mis à jour
output_csv_path

In [ ]:
from google.colab import files
import requests
import csv
import sys
from bs4 import BeautifulSoup

# Étape 1: Charger le fichier CSV
uploaded = files.upload()  # Téléchargez votre fichier CSV ici.

# Étape 2: Mettre à jour le chemin du fichier CSV avec le nom du fichier téléchargé
input_csv_path = '/content/step1-update.csv'  # Remplacez par le nom de votre fichier téléchargé
output_csv_path = '/content/final-step1.csv'

# Augmenter la limite maximale de taille de champ
csv.field_size_limit(sys.maxsize)

# Étape 3: Fonction pour extraire les détails de Wikipédia
def get_wikipedia_data_from_url(url):
    response = requests.get(url)

    if response.status_code == 200:
        # Utiliser BeautifulSoup pour extraire le contenu de la page Wikipedia
        soup = BeautifulSoup(response.text, "html.parser")

        # Extraire le titre de la page
        page_title = soup.find("h1", {"id": "firstHeading"}).text if soup.find("h1", {"id": "firstHeading"}) else "N/A"

        # Extraire le texte de l'article (principalement à partir de balises <p>)
        paragraphs = soup.find_all("p")
        article_extract = " ".join([p.text for p in paragraphs])
        article_extract = article_extract.strip()

        # Calcul du nombre de mots dans l'extrait
        word_count = len(article_extract.split()) if article_extract else 0

        # Extraire les liens internes (balises <a> dans le corps de l'article)
        internal_links = []
        for link in soup.find_all("a", href=True):
            href = link.get("href")
            if href.startswith("/wiki/") and not href.startswith("/wiki/Special:"):
                internal_links.append(link.text.strip())
        internal_links_str = "; ".join(internal_links)

        # Extraire les catégories (depuis la section des catégories)
        categories = []
        for category in soup.find_all("div", {"id": "mw-normal-catlinks"}):
            for link in category.find_all("a"):
                categories.append(link.text.strip())
        categories_str = "; ".join(categories)

        # Extraire les images
        images = []
        for img in soup.find_all("img"):
            images.append(img.get("src"))
        images_str = "; ".join(images)

        # Retourner les données sous forme de dictionnaire
        return {
            "page_title": page_title,
            "page_url": url,
            "article_extract": article_extract,
            "word_count": word_count,
            "internal_links": internal_links_str,
            "categories": categories_str,
            "images": images_str
        }
    else:
        print(f"Erreur lors de la récupération de la page : {url}")
        return None

# Étape 4: Lire le fichier CSV et appliquer la fonction à chaque URL
with open(input_csv_path, mode='r', encoding='utf-8') as infile, open(output_csv_path, mode='w', newline='', encoding='utf-8') as outfile:
    reader = csv.DictReader(infile)
    fieldnames = ["page_title", "page_url", "article_extract", "word_count", "internal_links", "categories", "images"]
    writer = csv.DictWriter(outfile, fieldnames=fieldnames)

    # Écrire les en-têtes dans le fichier de sortie
    writer.writeheader()

    # Appliquer la fonction à chaque ligne
    for row in reader:
        url = row['page_url']
        if url and url != "N/A":
            wikipedia_data = get_wikipedia_data_from_url(url)
            if wikipedia_data:
                writer.writerow(wikipedia_data)

# Étape 5: Télécharger le fichier CSV enrichi
files.download(output_csv_path)

Saving step1-update.csv to step1-update (2).csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Now that we have the Data "final-step1.csv", we use it to generate the concise text in another column using OpenAI API

In [ ]:
from openai import OpenAI

# Pass the API key directly to the OpenAI constructor
# Load the API key from the text file
with open('/content/api_key.txt', 'r') as file:
    api_key = file.read().strip()

# Initialize the OpenAI client using the loaded API key
client = OpenAI(api_key=api_key)



completion = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello!"}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='Hello! How can I assist you today?', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)


In [ ]:





completion = client.chat.completions.create(
    model = "gpt-4o",
    messages = [
        {"role": "system", "content": "You are a helpful assistant who generates child-friendly, concise summaries of stories."},
        {"role": "user", "content": (
            f"Générer dans une autre colonne l'histoire du conte / de l'oeuvre '{title}' "
            f"et le raconter d'une façon à ce qu'on s'adresse à l'enfant de manière pas trop longue. "
            f"Bien narrer de manière adaptée pour un enfant de bas âge. Une explication concise.\n\n"
            f"Voici l'extrait de la page Wikipédia :\n\n{text}"
        )}
    ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Il était une fois un petit prince qui vivait sur une toute petite planète avec une rose. Un jour, il décida de partir explorer l'univers et rencontra plein de personnages intéressants, comme un roi, un homme d'affaires, et un allumeur de réverbères. Chaque rencontre lui apprend quelque chose de nouveau.\n\nEnsuite, il arrive sur Terre, où il devient ami avec un aviateur perdu dans le désert. Le petit prince découvre aussi un renard, qui lui révèle l'importance de l'amitié et du fait d'apprivoiser quelqu'un. Grâce à ses aventures, le petit prince comprend que l'amour et les liens sont ce qui compte le plus dans la vie. \n\nÀ la fin, le petit prince décide de retourner sur sa planète pour être avec sa rose, mais ses enseignements sur l'amitié et l'amour restent gravés dans le cœur de l'aviateur et de tous ceux qui entendent son histoire.", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)


# II - Dataset Creation : ENGLISH DATASET

In [ ]:
import pandas as pd

# List of titles
titles_list = [
    "Alice's Adventures in Wonderland",
    "Peter Pan",
    "Robin Hood",
    "Mulan",
    "Pocahontas",
    "The Jungle Book",
    "Bambi",
    "The Lion King",
    "Winnie-the-Pooh",
    "The Princess and the Frog",
    "Frozen",
    "Moana",
    "Snow-White and Rose-Red",
    "Ali Baba and the Forty Thieves",
    "Sinbad the Sailor",
    "Hop-o'-My-Thumb",
    "Donkeyskin",
    "The Adventures of Tintin",
    "Asterix and Obelix",
    "The Wizard of Oz",
    "Oliver Twist",
    "The Twelve Dancing Princesses",
    "The Pied Piper of Hamelin",
    "The Princess and the Pea",
    "The Bremen Town Musicians",
    "Aladdin",
    "Anastasia",
    "The Little Match Girl",
    "The Wild Swans",
    "Madame Bovary",
    "Heidi",
    "The Nightmare Before Christmas",
    "Wallace and Gromit",
    "Chicken Run",
    "Madagascar",
    "Kung Fu Panda",
    "Toy Story",
    "Finding Nemo",
    "The Incredibles",
    "Shrek",
    "Cars",
    "Ratatouille",
    "Monsters, Inc.",
    "Coco",
    "The Adventures of Tom Sawyer",
    "Little Women",
    "Mary Poppins",
    "The Sound of Music",
    "Pippi Longstocking",
    "Pete's Dragon",
    "Ferdinand",
    "The Little Prince",
    "The Black Cauldron",
    "Balto",
    "Spirit: Stallion of the Cimarron",
    "The Aristocats",
    "The Fox and the Child",
    "Animals of Farthing Wood",
    "The Rescuers",
    "Tinker Bell",
    "Brother Bear",
    "Lilo & Stitch",
    "Rio",
    "Brave",
    "Horton Hears a Who!",
    "The Smurfs",
    "Zootopia",
    "Trolls",
    "The Lion Guard",
    "The Legend of Korra",
    "Le Petit Nicolas",
    "Alice Through the Looking Glass",
    "Charlie and the Chocolate Factory",
    "The Iron Giant",
    "Mia and the Migoo",
    "Wolf Children",
    "Ponyo",
    "My Neighbor Totoro",
    "Kiki's Delivery Service",
    "Spirited Away",
    "Princess Mononoke",
    "Howl's Moving Castle",
    "Nausicaä of the Valley of the Wind",
    "Children of the Sea",
    "Castle in the Sky",
    "Coraline",
    "Anastasia",
    "Ernest & Celestine",
    "Paddington",
    "The Gruffalo",
    "Where the Wild Things Are",
    "The King and the Mockingbird",
    "Kirikou and the Sorceress",
    "Azur & Asmar",
    "The NeverEnding Story",
    "The Great Gilly Hopkins",
    "A Christmas Carol",
    "The Little Mermaid",
    "The Three Robbers",
    "The Fox and the Maple"
]

# Function to create a formatted Wikipedia link part from the translated title
def create_formatted_title(title):
    # Replace each single quote with %27 and spaces with underscores
    formatted_title = title.replace("'", "%27").replace(" ", "_")
    return formatted_title

# Generate formatted titles
formatted_titles = [create_formatted_title(title) for title in titles_list]

# Create the full URLs from the formatted titles
english_urls = [f"https://en.wikipedia.org/wiki/{formatted_title}" for formatted_title in formatted_titles]

# Assume df is already loaded, add the english_urls as a new column
input_csv_path = '/content/final-step1.csv'
df = pd.read_csv(input_csv_path)

# Add the new column with the English Wikipedia URLs
df['english_url'] = english_urls

# Save the updated dataframe to a new CSV file
output_csv_path = '/content/final-step1-with-english-url.csv'
df.to_csv(output_csv_path, index=False)

print(f"CSV file updated and saved successfully: {output_csv_path}")

CSV file updated and saved successfully: /content/final-step1-with-english-url.csv


In [ ]:
from google.colab import files
import requests
import csv
import sys
from bs4 import BeautifulSoup

# Étape 1: Charger le fichier CSV
uploaded = files.upload()  # Téléchargez votre fichier CSV ici.

# Étape 2: Mettre à jour le chemin du fichier CSV avec le nom du fichier téléchargé
input_csv_path = '/content/final-step1-with-english-url.csv'  # Remplacez par le nom de votre fichier téléchargé
output_csv_path = '/content/final-step1-with-english-text.csv'

# Augmenter la limite maximale de taille de champ
csv.field_size_limit(sys.maxsize)

# Étape 3: Fonction pour extraire les détails de Wikipédia
def get_wikipedia_data_from_url(url):
    response = requests.get(url)

    if response.status_code == 200:
        # Utiliser BeautifulSoup pour extraire le contenu de la page Wikipedia
        soup = BeautifulSoup(response.text, "html.parser")

        # Extraire le titre de la page
        page_title = soup.find("h1", {"id": "firstHeading"}).text if soup.find("h1", {"id": "firstHeading"}) else "N/A"

        # Extraire le texte de l'article (principalement à partir de balises <p>)
        paragraphs = soup.find_all("p")
        article_extract = " ".join([p.text for p in paragraphs])
        article_extract = article_extract.strip()

        # Calcul du nombre de mots dans l'extrait
        word_count = len(article_extract.split()) if article_extract else 0

        # Extraire les liens internes (balises <a> dans le corps de l'article)
        internal_links = []
        for link in soup.find_all("a", href=True):
            href = link.get("href")
            if href.startswith("/wiki/") and not href.startswith("/wiki/Special:"):
                internal_links.append(link.text.strip())
        internal_links_str = "; ".join(internal_links)

        # Extraire les catégories (depuis la section des catégories)
        categories = []
        for category in soup.find_all("div", {"id": "mw-normal-catlinks"}):
            for link in category.find_all("a"):
                categories.append(link.text.strip())
        categories_str = "; ".join(categories)

        # Extraire les images
        images = []
        for img in soup.find_all("img"):
            images.append(img.get("src"))
        images_str = "; ".join(images)

        # Retourner les données sous forme de dictionnaire
        return {
            "en_page_title": page_title,
            "en_page_url": url,
            "en_article_extract": article_extract,
            "en_word_count": word_count,
            "en_internal_links": internal_links_str,
            "en_categories": categories_str,
            "en_images": images_str
        }
    else:
        print(f"Erreur lors de la récupération de la page : {url}")
        return None

# Étape 4: Lire le fichier CSV et appliquer la fonction à chaque URL
with open(input_csv_path, mode='r', encoding='utf-8') as infile, open(output_csv_path, mode='w', newline='', encoding='utf-8') as outfile:
    reader = csv.DictReader(infile)
    fieldnames = ["en_page_title", "en_page_url", "en_article_extract", "en_word_count", "en_internal_links", "en_categories", "en_images"]
    writer = csv.DictWriter(outfile, fieldnames=fieldnames)

    # Écrire les en-têtes dans le fichier de sortie
    writer.writeheader()

    # Appliquer la fonction à chaque ligne
    for row in reader:
        url = row['english_url']
        if url and url != "N/A":
            wikipedia_data = get_wikipedia_data_from_url(url)
            if wikipedia_data:
                writer.writerow(wikipedia_data)

# Étape 5: Télécharger le fichier CSV enrichi
files.download(output_csv_path)

Saving final-step1-with-english-url.csv to final-step1-with-english-url (1).csv
Erreur lors de la récupération de la page : https://en.wikipedia.org/wiki/The_Fox_and_the_Maple


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:

# Define the function to rename the first columns
def rename_columns(input_csv_path, output_csv_path):
    # Read the CSV file using pandas
    df = pd.read_csv(input_csv_path)

    # Define the new fieldnames for the first columns
    new_fieldnames = ["fr_page_title", "fr_page_url", "fr_article_extract", "fr_word_count", "fr_internal_links", "fr_categories", "fr_images"]

    # Rename the first columns with the new fieldnames
    for i, new_name in enumerate(new_fieldnames):
        df.rename(columns={df.columns[i]: new_name}, inplace=True)

    # Save the updated dataframe to a new CSV file
    df.to_csv(output_csv_path, index=False)

    print(f"CSV file with renamed columns saved successfully: {output_csv_path}")

# File paths
input_csv_path = '/content/fr_data.csv'
output_csv_path = '/content/new_fr_data.csv'

# Call the function to rename the columns and save the updated CSV
rename_columns(input_csv_path, output_csv_path)

CSV file with renamed columns saved successfully: /content/new_fr_data.csv


### Merge the Fr_dataset to En_dataset

In [ ]:
import pandas as pd

# File paths for the input CSV files
fr_data_path = '/content/new_fr_data.csv'
en_data_path = '/content/en_data.csv'

# Read the French and English CSV files into dataframes
df_fr = pd.read_csv(fr_data_path)
df_en = pd.read_csv(en_data_path)

# Merge the dataframes on the specified columns
merged_df = pd.merge(df_fr, df_en, how='inner', left_on='english_url', right_on='en_page_url')

# Save the merged dataframe to a new CSV file
output_csv_path = '/content/merged_fr_en_data.csv'
merged_df.to_csv(output_csv_path, index=False)

print(f"Merged CSV file saved successfully: {output_csv_path}")

Merged CSV file saved successfully: /content/merged_fr_en_data.csv


## FUNCTION FOR GENERATING RELEVANT FORMAT

In [ ]:
import pandas as pd

# Read the CSV file using pandas
input_csv_path = '/content/merged_fr_en_data.csv'
df = pd.read_csv(input_csv_path)
num_rows = len(df)  # Set the number of rows to process as the total number of rows

# Define a function to read the dataset and extract the columns as lists
def read_dataset(df, num_rows=5):
    # Extract the "article_extract" and "page_title" for the first `num_rows` rows
    text_list = df["en_article_extract"].tolist()[:num_rows]
    title_list = df["en_page_title"].tolist()[:num_rows]

    return title_list, text_list

# Read the dataset and get lists for the first `num_rows` rows
titles, texts = read_dataset(df, num_rows=num_rows)

# Print the extracted lists to verify
print(f"Titles: {titles}")
print(f"Texts: {texts}")

Titles: ["Alice's Adventures in Wonderland", 'Peter Pan', 'Robin Hood', 'Hua Mulan', 'Pocahontas', 'The Jungle Book', 'Bambi', 'The Lion King', 'Winnie-the-Pooh', 'The Princess and the Frog', 'Frozen', 'Moana', 'Snow-White and Rose-Red', 'Ali Baba and the Forty Thieves', 'Sinbad the Sailor', "Hop-o'-My-Thumb", 'Donkeyskin', 'The Adventures of Tintin', 'Asterix', 'The Wizard of Oz', 'Oliver Twist', 'The Twelve Dancing Princesses', 'Pied Piper of Hamelin', 'The Princess and the Pea', 'Town Musicians of Bremen', 'Aladdin', 'Anastasia', 'Anastasia', 'The Little Match Girl', 'The Wild Swans', 'Madame Bovary', 'Heidi', 'The Nightmare Before Christmas', 'Wallace & Gromit', 'Chicken Run', 'Madagascar', 'Kung Fu Panda', 'Toy Story', 'Finding Nemo', 'The Incredibles', 'Shrek', 'Car', 'Ratatouille', 'Monsters, Inc.', 'Coco', 'The Adventures of Tom Sawyer', 'Little Women', 'Mary Poppins', 'The Sound of Music', 'Pippi Longstocking', "Pete's Dragon", 'Ferdinand', 'The Little Prince', 'The Black Caul

In [ ]:
#pip install tiktoken

In [ ]:
import tiktoken
import time


# List to store the generated summaries
summaries = []
token_counts = []


# Initialize tiktoken encoder for the "gpt-4o" model
enc = tiktoken.encoding_for_model("gpt-4o")


# Loop over each title and text and generate summaries using OpenAI API
for i in range(num_rows):
    title = titles[i]
    extract = texts[i]

    # Create the messages format expected by ChatCompletion
    messages = [
        {"role": "system", "content": "You are a helpful assistant who generates child-friendly, concise summaries of stories."},
        {"role": "user", "content": (
            f"Générer dans une autre colonne l'histoire du conte / de l'oeuvre '{title}' "
            f"et le raconter d'une façon à ce qu'on s'adresse à l'enfant de manière pas trop longue. "
            f"Bien narrer de manière adaptée pour un enfant de bas âge. Une explication concise.\n\n"
            f"Voici l'extrait de la page Wikipédia :\n\n{extract}"
        )}
    ]

    # Token count for the messages
    total_tokens = 0
    for message in messages:
        message_tokens = len(enc.encode(message["content"]))
        total_tokens += message_tokens

    # Print the number of tokens used for the messages
    print(f"Total tokens for row {i + 1}: {total_tokens}")


    completion = client.chat.completions.create(
        model = "gpt-4o",
        messages = messages
    )


    print(completion.choices[0].message)
    # Extract the generated summary
    summary = completion.choices[0].message.content.strip()
    summaries.append(summary)

    token_counts.append(total_tokens)

    # Add a delay after processing every 10 rows
    #if (i + 1) % 5 == 0:
    #    print("Processed 5 rows, waiting for 1 minute to avoid rate limits...")
    #    time.sleep(80)



Total tokens for row 1: 6226
ChatCompletionMessage(content='Voici une version simplifiée de l\'histoire d\'"Alice au pays des merveilles" pour les jeunes enfants :\n\nUn jour, une petite fille nommée Alice se promenait au bord d\'une rivière lorsqu\'elle aperçut un Lapin Blanc qui semblait très pressé. Curieuse, elle le suivit et tomba dans un terrier de lapin qui l\'amena dans un monde magique et étrange appelé le pays des merveilles. Là-bas, tout était à l\'envers, et les animaux pouvaient parler comme des gens !\n\nAlice fit la rencontre de nombreux personnages fantastiques : un gros chat souriant appelé le Chat du Cheshire, un Chapelier Fou qui aime le thé, et une Reine de Cœurs qui adore donner des ordres de couper la tête à tout le monde ! \n\nAlice grandit et rétrécit en mangeant et buvant des choses étranges. Elle participa même à une partie de croquet bizarre où les maillets étaient des flamants roses et les balles des hérissons !\n\nFinalement, Alice se retrouva face à la Rei

In [ ]:
output_csv_path = '/content/final-step2-openai.csv'

# Add the summaries as a new column to the dataframe
df['concise_story'] = pd.Series(summaries, index=df.index[:num_rows])

# Save the updated dataframe to a new CSV file
df.to_csv(output_csv_path, index=False)

# Output confirmation
print(f"CSV file enriched successfully and saved to: {output_csv_path}")

CSV file enriched successfully and saved to: /content/final-step2-openai.csv


### Load & Filter the Extracted data

In [ ]:
import pandas as pd
import re


output_openai = '/content/final-step2-openai.csv'

dff = pd.read_csv(output_openai)



# Define a function to remove characters like [\d+]
def remove_bracketed_numbers(text):
    return re.sub(r'\[\d+\]', '', text)

# Apply the function to the "en_article_extract" column
dff["en_article_extract"] = dff["en_article_extract"].apply(remove_bracketed_numbers)
dff["fr_article_extract"] = dff["fr_article_extract"].apply(remove_bracketed_numbers)

dff

fr_page_title  \
0    Les Aventures d'Alice au pays des merveilles   
1                                       Peter Pan   
2                                  Robin des Bois   
3                                           Mulan   
4                                      Pocahontas   
..                                            ...   
96                     L'Histoire sans fin (film)   
97                     La Fabuleuse Gilly Hopkins   
98                               Un chant de Noël   
99                  Le Secret de la Petite Sirène   
100                     Les Trois Brigands (film)   

                                           fr_page_url  \
0    https://fr.wikipedia.org/wiki/Alice_au_Pays_de...   
1              https://fr.wikipedia.org/wiki/Peter_Pan   
2         https://fr.wikipedia.org/wiki/Robin_des_Bois   
3                  https://fr.wikipedia.org/wiki/Mulan   
4             https://fr.wikipedia.org/wiki/Pocahontas   
..                                                 ...   
96   https://fr.wikipedia.org/wiki/L%27Histoire_san...   
97   https://fr.wikipedia.org/wiki/La_Fabuleuse_Gil...   
98      https://fr.wikipedia.org/wiki/Un_Chant_de_Noël   
99   https://fr.wikipedia.org/wiki/Le_Secret_de_la_...   
100  https://fr.wikipedia.org/wiki/Les_Trois_Brigan...   

                                    fr_article_extract  fr_word_count  \
0    « Alice au pays des merveilles » redirige ici....           2706   
1    Pour les articles homonymes, voir Peter Pan (h...           2996   
2    Pour les articles homonymes, voir Robin des Bo...           1830   
3    Série Classiques d'animation Disney Hercule(19...           2238   
4    Pour les articles homonymes, voir Pocahontas (...           2284   
..                                                 ...            ...   
96   Pour les articles homonymes, voir L'Histoire s...           1355   
97   Cet article est une ébauche concernant un film...            131   
98   A Christmas Carol Vous lisez un « article de q...          13449   
99   Pour les articles homonymes, voir La Petite Si...            856   
100  Pour les articles homonymes, voir Les Trois Br...            242   

                                     fr_internal_links  \
0    Accueil; Portails thématiques; Article au hasa...   
1    Accueil; Portails thématiques; Article au hasa...   
2    Accueil; Portails thématiques; Article au hasa...   
3    Accueil; Portails thématiques; Article au hasa...   
4    Accueil; Portails thématiques; Article au hasa...   
..                                                 ...   
96   Accueil; Portails thématiques; Article au hasa...   
97   Accueil; Portails thématiques; Article au hasa...   
98   Accueil; Portails thématiques; Article au hasa...   
99   Accueil; Portails thématiques; Article au hasa...   
100  Accueil; Portails thématiques; Article au hasa...   

                                         fr_categories  \
0    Catégories; Alice au pays des merveilles; Roma...   
1    Catégories; Peter Pan; Œuvre de littérature d'...   
2    Catégories; Robin des Bois; Archer de fiction;...   
3    Catégories; Film américain sorti en 1998; Film...   
4    Catégories; Dirigeant politique du XVIIe siècl...   
..                                                 ...   
96   Catégories; L'Histoire sans fin; Film réalisé ...   
97   Catégories; Film américain sorti en 2015; Comé...   
98   Catégories; Un chant de Noël; Noël dans la lit...   
99   Catégories; Film américain sorti en 2008; Film...   
100  Catégories; Comédie allemande; Film allemand s...   

                                             fr_images  \
0    /static/images/icons/wikipedia.png; /static/im...   
1    /static/images/icons/wikipedia.png; /static/im...   
2    /static/images/icons/wikipedia.png; /static/im...   
3    /static/images/icons/wikipedia.png; /static/im...   
4    /static/images/icons/wikipedia.png; /static/im...   
..                                                 ...   
96   /static/images/icons

In [ ]:
titles, texts = read_dataset(dff, num_rows=num_rows)

### Generating the narrative story

In [ ]:
import tiktoken
import random
import time


# List to store the generated summaries
en_summaries = []
token_counts = []


# Initialize tiktoken encoder for the "gpt-4o" model
enc = tiktoken.encoding_for_model("gpt-4o")


# Loop over each title and text and generate summaries using OpenAI API
for i in range(num_rows):
    title = titles[i]
    extract = texts[i]

    # Create the messages format expected by ChatCompletion
    """
    messages = [
    {"role": "system", "content": "You are a helpful assistant who creates child-friendly, engaging versions of stories."},
    {"role": "user", "content": (
        f"Reformulate the story of '{title}' into a children's tale, telling it in a way that is engaging and easy for young children to understand. "
        f"Make sure the story feels like an enchanting bedtime tale, with a simple and friendly tone suitable for young children.\n\n"
        f"Here is an extract from the Wikipedia page:\n\n{extract}"
    )}
    ]
    """
    # Define a list of possible opening phrases to vary the beginning of the story
    opening_phrases = [
      "Long ago in a distant land,",
      "There was a time when",
      "In a magical kingdom,",
      "In a faraway place,",
      "Not too long ago,",
      "Deep in the enchanted forest,",
      "High on a mountain top,",
      "In a tiny village,",
      "In a beautiful land of wonder,",
      "In the middle of a bustling town,",
      "Once upon a time,"
    ]

    # Select a random opening phrase for variety
    opening = random.choice(opening_phrases)
    #print(opening)

    messages = [
        {"role": "system", "content": "You are a helpful assistant who creates child-friendly, engaging versions of stories. Do not start the story with 'Once upon a time'. Use creative and different beginnings instead."},
        {"role": "user", "content": (
            f"Reformulate the story of '{title}' into a children's tale, telling it in a way that is engaging and easy for young children to understand. "
            f"Make sure the story feels like an enchanting bedtime tale, with a simple and friendly tone suitable for young children. Start with this: '{opening}'\n\n"
            f"Here is an extract from the Wikipedia page:\n\n{extract}"
        )}
    ]


    # Token count for the messages
    total_tokens = 0
    for message in messages:
        message_tokens = len(enc.encode(message["content"]))
        total_tokens += message_tokens

    # Print the number of tokens used for the messages
    print(f"Total tokens for row {i + 1}: {total_tokens}")


    completion = client.chat.completions.create(
        model = "gpt-4o",
        messages = messages
    )


    print(completion.choices[0].message)
    # Extract the generated summary
    en_summary = completion.choices[0].message.content.strip()
    en_summaries.append(en_summary)

    token_counts.append(total_tokens)

    # Add a delay after processing every 10 rows
    #if (i + 1) % 5 == 0:
    #    print("Processed 5 rows, waiting for 1 minute to avoid rate limits...")
    #    time.sleep(80)



Total tokens for row 1: 5973
ChatCompletionMessage(content='Deep in the enchanted forest, there was a curious girl named Alice. One warm afternoon, as she wandered beneath the leafy trees, she spotted something quite peculiar—a White Rabbit wearing a waistcoat and peering anxiously at a pocket watch. \n\nUtterly fascinated, Alice trailed after the rabbit, who was muttering about being late. He led her to a rabbit hole, and before Alice knew it, she was falling, falling, falling, until she gently landed in a wondrous and whimsical world.\n\nIn this topsy-turvy place, Alice found herself in a tiny room with a little door. Through its keyhole, she glimpsed the loveliest garden and longed to explore it. But there was a problem—the door was far too small for Alice to fit through!\n\nThen Alice noticed a bottle labeled "Drink me." With a sip, she magically shrank to the perfect size to slip through the door but realized she\'d forgotten the key on the table! Distressed, Alice ate a cake labe

In [ ]:
en_output_csv_path = '/content/final-step3-openai.csv'

# Add the summaries as a new column to the dataframe
dff['en_concise_story'] = pd.Series(en_summaries, index=df.index[:num_rows])

# Save the updated dataframe to a new CSV file
dff.to_csv(en_output_csv_path, index=False)

# Output confirmation
print(f"CSV file enriched successfully and saved to: {en_output_csv_path}")

CSV file enriched successfully and saved to: /content/final-step3-openai.csv


# III - Data Augmentation

In [ ]:
import pandas as pd

df = pd.read_csv("/content/final-step3-openai-tuned.csv")
df


fr_page_title  \
0    Les Aventures d'Alice au pays des merveilles   
1                                       Peter Pan   
2                                  Robin des Bois   
3                                           Mulan   
4                                      Pocahontas   
..                                            ...   
96                     L'Histoire sans fin (film)   
97                     La Fabuleuse Gilly Hopkins   
98                               Un chant de Noël   
99                  Le Secret de la Petite Sirène   
100                     Les Trois Brigands (film)   

                                           fr_page_url  \
0    https://fr.wikipedia.org/wiki/Alice_au_Pays_de...   
1              https://fr.wikipedia.org/wiki/Peter_Pan   
2         https://fr.wikipedia.org/wiki/Robin_des_Bois   
3                  https://fr.wikipedia.org/wiki/Mulan   
4             https://fr.wikipedia.org/wiki/Pocahontas   
..                                                 ...   
96   https://fr.wikipedia.org/wiki/L%27Histoire_san...   
97   https://fr.wikipedia.org/wiki/La_Fabuleuse_Gil...   
98      https://fr.wikipedia.org/wiki/Un_Chant_de_Noël   
99   https://fr.wikipedia.org/wiki/Le_Secret_de_la_...   
100  https://fr.wikipedia.org/wiki/Les_Trois_Brigan...   

                                    fr_article_extract  fr_word_count  \
0    « Alice au pays des merveilles » redirige ici....           2706   
1    Pour les articles homonymes, voir Peter Pan (h...           2996   
2    Pour les articles homonymes, voir Robin des Bo...           1830   
3    Série Classiques d'animation Disney Hercule(19...           2238   
4    Pour les articles homonymes, voir Pocahontas (...           2284   
..                                                 ...            ...   
96   Pour les articles homonymes, voir L'Histoire s...           1355   
97   Cet article est une ébauche concernant un film...            131   
98   A Christmas Carol Vous lisez un « article de q...          13449   
99   Pour les articles homonymes, voir La Petite Si...            856   
100  Pour les articles homonymes, voir Les Trois Br...            242   

                                     fr_internal_links  \
0    Accueil; Portails thématiques; Article au hasa...   
1    Accueil; Portails thématiques; Article au hasa...   
2    Accueil; Portails thématiques; Article au hasa...   
3    Accueil; Portails thématiques; Article au hasa...   
4    Accueil; Portails thématiques; Article au hasa...   
..                                                 ...   
96   Accueil; Portails thématiques; Article au hasa...   
97   Accueil; Portails thématiques; Article au hasa...   
98   Accueil; Portails thématiques; Article au hasa...   
99   Accueil; Portails thématiques; Article au hasa...   
100  Accueil; Portails thématiques; Article au hasa...   

                                         fr_categories  \
0    Catégories; Alice au pays des merveilles; Roma...   
1    Catégories; Peter Pan; Œuvre de littérature d'...   
2    Catégories; Robin des Bois; Archer de fiction;...   
3    Catégories; Film américain sorti en 1998; Film...   
4    Catégories; Dirigeant politique du XVIIe siècl...   
..                                                 ...   
96   Catégories; L'Histoire sans fin; Film réalisé ...   
97   Catégories; Film américain sorti en 2015; Comé...   
98   Catégories; Un chant de Noël; Noël dans la lit...   
99   Catégories; Film américain sorti en 2008; Film...   
100  Catégories; Comédie allemande; Film allemand s...   

                                             fr_images  \
0    /static/images/icons/wikipedia.png; /static/im...   
1    /static/images/icons/wikipedia.png; /static/im...   
2    /static/images/icons/wikipedia.png; /static/im...   
3    /static/images/icons/wikipedia.png; /static/im...   
4    /static/images/icons/wikipedia.png; /static/im...   
..                                                 ...   
96   /static/images/icons

In [ ]:
existing_list = df["en_page_title"]

In [ ]:
new_list = []
new_list.extend([
    "The Adventure of the Glowing Giant",
    "The Hedgehog's Starry Surprise",
    "The Magic Book and the Story Sprites",
    "The Tale of the Rainbow Lion",
    "The Night the Ocean Sang",
    "The Enchanted Paper Plane",
    "The Tale of the Twinkling Treetops",
    "The Bear's Nighttime Balloon Ride",
    "The Dancing Blossoms of Enchanted Grove",
    "The Whispering Stars of the Midnight Sky",
    "The Crystal Bunny and the Golden Carrot",
    "The Giggling Galaxy Adventure",
    "The Tale of the Singing Seahorse",
    "The Adventures of the Starry Squirrel",
    "The Forest of Laughing Mushrooms",
    "The Adventure of the Silver Shell",
    "The Giraffe Who Found a Rainbow",
    "The Journey of the Lost Snowflake",
    "The Tale of the Crystal Canoe",
    "The Mermaid's Glittering Necklace",
    "The Garden of Everlasting Fireflies",
    "The Adventure of the Floating Lanterns",
    "The Penguin's Winter Wonderland",
    "The Little Duck's Moonlight Dance",
    "The Tale of the Glittering Gazelle",
    "The Magical Meadow of Talking Butterflies",
    "The Dolphin's Twinkling Treasure",
    "The Day the Hills Were Covered in Flowers",
    "The Fairy Who Painted the Sky",
    "The Tale of the Tiny Teardrop",
    "The Owl's Midnight Melody",
    "The Adventure of the Laughing Lily",
    "The Peacock's Rainbow Plumage",
    "The Story of the Shimmering Snowman",
    "The Day the Waterfall Turned Gold",
    "The Tale of the Whispering Unicorn",
    "The Fireflies' Festival of Stars",
    "The Secret of the Sapphire Stream",
    "The Adventure of the Singing Hills",
    "The Tale of the Fluttering Dragonfly",
    "The Lost Kingdom of Cotton Candy",
    "The Little Panda's Starry Night",
    "The Magical Map to Glitter Gorge",
    "The Fox Who Played the Moonlight Harp",
    "The Midnight Parade of Mischievous Pixies",
    "The Tale of the Velvet Violin",
    "The Giggling Giants of Jellybean Hill",
    "The Story of the Dreaming Trees",
    "The Snow Leopard's Glittering Crown",
    "The Panda's Glow-in-the-Dark Adventure",
    "The Enchanted Ice Skating Pond",
    "The Tale of the Shimmering Shield",
    "The Rabbit Who Rode a Rainbow",
    "The Adventure of the Golden Windchimes",
    "The Butterfly's Crystal Wings",
    "The Tale of the Glimmering Grapes",
    "The Midnight Dance of the Firebirds",
    "The Dragon Who Collected Pebbles",
    "The Whispering Shell and the Talking Crab",
    "The Panda's Picnic Under the Stars",
    "The Day the Forest Became a Painting",
    "The Adventure of the Mischievous Map",
    "The Peacock's Feathered Magic",
    "The Tale of the Secret Stream",
    "The Little Bear's Nighttime Adventure",
    "The Giggles of the Glittering Goblin",
    "The Adventure of the Moonlit Maze",
    "The Tale of the Sparkling Spring",
    "The Fairy's Firefly Lantern",
    "The Day the Stars Became Ribbons",
    "The Panda's Secret Ice Cream Feast",
    "The Tale of the Blooming Bellflower",
    "The Adventure of the Singing Stream",
    "The Snowy's Tale of Adventure",
    "The Moonlight Pony's Starry Path",
    "The Tale of the Glittering Crown",
    "The Adventure of the Wandering Star",
    "The Day the Mountains Learned to Dance",
    "The Magical Ribbon and the Silly Swan",
    "The Tale of the Sunflower's Secret",
    "The Gigantic Ice Cream Mountain",
    "The Little Fox and the Twinkling Gem",
    "The Day the Ocean Whispered Secrets",
    "The Unicorn's Glittering Meadow",
    "The Tale of the Singing Stones",
    "The Adventure of the Crystal Balloon",
    "The Bunny's Bubblegum Adventure",
    "The Giggleberries of Glitterland",
    "The Tale of the Glistening Gargoyle",
    "The Rainbow Sea and the Golden Pearl",
    "The Bear's Adventures with the Stars",
    "The Crystal Kangaroo and the Cotton Cloud",
    "The Day the Crickets Sang a Symphony",
    "The Magical Garden of Hummingbirds",
    "The Tale of the Enchanted Acorn",
    "The Fox's Feathered Fortune",
    "The Golden Marshmallow Mystery",
    "The Midnight Whisper of the Willow",
    "The Talking Shell and the Sunset",
    "The Panda's Glowworm Gala",
    "The Tale of the Laughing Leopard",
    "The Bear and the Enchanted Violin",
    "The Fairy's Shimmering Tutu",
    "The Owl's Moonlit Picnic",
    "The Day the Sun Fell Asleep",
    "The Unicorn's Secret Wish",
    "The Tale of the Glittering Goblet",
    "The Adventures of the Rainbow Trout",
    "The Crystal Cat and the Emerald Feather",
    "The Giggling Goblin's Treasure Map",
    "The Snowy River of Shimmering Lights",
    "The Fairy's Sparkling Pathway",
    "The Day the Trees Learned to Talk",
    "The Panda's Magical Blanket",
    "The Tale of the Golden Star",
    "The Glow-in-the-Dark Narwhal",
    "The Peacock Who Lost His Colors",
    "The Day the Fairies Rode the Wind",
    "The Tale of the Shining Seahorse",
    "The Crystal Gardens of Starry Hollow",
    "The Bear's Cotton Candy Adventure",
    "The Night the River Played Music",
    "The Owl's Glittering Treasure Hunt",
    "The Day the Rain Turned to Glitter",
    "The Tale of the Rainbow Woods",
    "The Adventure of the Sparkling Clover",
    "The Gigantic Glowworm",
    "The Day the Moon Became a Cookie",
    "The Whispering Birds of Twilight",
    "The Crystal Frog's Magical Pond",
    "The Tale of the Singing Hills",
    "The Adventure of the Enchanted Clock",
    "The Bunny and the Talking Teapot",
    "The Day the Stars Told Stories",
    "The Owl's Enchanted Feather",
    "The Tale of the Laughing Fox",
    "The Midnight Fairies' Lantern Dance",
    "The Adventure of the Magic Beanstalk",
    "The Giggles of the Golden Geese",
    "The Little Bear's Day at Glitter Lake",
    "The Snow Leopard's Midnight Journey",
    "The Unicorn's Tale of Shining Stars",
    "The Tale of the Singing Waterfall",
    "The Adventure of the Golden Whistle",
    "The Day the Forest Turned into Candy",
    "The Fox's Giggling Adventure",
    "The Panda's Glittering Garland",
    "The Tale of the Crystal Cave",
    "The Dancing Unicorn and the Star Tree",
    "The Day the River Glowed Pink",
    "The Adventure of the Enchanted Oak",
    "The Laughing Leprechaun's Secret",
    "The Owl's Magical Midnight Ride",
    "The Tale of the Whispering Rainbows",
    "The Bear and the Sparkling Stars",
    "The Crystal Swans of Moonbeam Lake",
    "The Fox's Snowy Journey",
    "The Tale of the Rainbow Sprites",
    "The Golden River Adventure",
    "The Dancing Blossoms of Rainbow Grove",
    "The Owl and the Midnight Treasure",
    "The Adventure of the Shimmering Lake",
    "The Panda's Fluffy Cloud Adventure",
    "The Tale of the Crystal Crown",
    "The Gigantic Gumball Mystery",
    "The Bunny Who Found the Rainbow",
    "The Fox's Floating Adventure",
    "The Tale of the Midnight Rose",
    "The Crystal Garden of Stars",
    "The Day the Rain Whispered Secrets",
    "The Unicorn's Sparkling Adventure",
    "The Little Bear's Gigantic Lollipop",
    "The Tale of the Singing Leaves",
    "The Adventure of the Glistening Brook",
    "The Golden Unicorn's Secret Meadow",
    "The Panda and the Shining Pearl",
    "The Midnight Sparkles of the River",
    "The Tale of the Gigantic Jellybean",
    "The Night the Stars Danced Together",
    "The Adventure of the Whispering Flowers",
    "The Story of the Magic Meadow",
    "The Panda's Rainbow Slide",
    "The Tale of the Singing Sky",
    "The Adventure of the Glittering Stream",
    "The Little Fox and the Star Dance",
    "The Tale of the Whispering Pebble",
    "The Dancing Trees of Crystal Forest",
    "The Adventure of the Moonlit Swing",
    "The Owl's Glittering Forest",
    "The Tale of the Magical Meadow",
    "The Gigantic Pumpkin Adventure",
    "The Midnight Crystals of Snowy Vale",
    "The Adventure of the Singing Stars",
    "The Little Bear's Floating Lantern",
    "The Tale of the Shining Gem",
    "The Panda's Magical Melon Adventure",
    "The Dancing Fairies of Glitter Glen",
    "The Adventure of the Rainbow Cloud",
    "The Tale of the Whistling Forest",
    "The Day the Clouds Turned to Candy",
    "The Fox's Snowy Forest Adventure",
    "The Adventure of the Enchanted Kite",
    "The Tale of the Golden Nightingale",
    "The Laughing Goblin's Shiny Secret",
    "The Adventure of the Crystal Forest",
    "The Day the Stars Shone Brightly",
    "The Bear's Rainbow Slide Adventure",
    "The Tale of the Gigantic Mushroom",
    "The Panda's Whispering Stream Adventure",
    "The Giggles of the Crystal Fairies",
    "The Tale of the Moonlit Bridge"
])

new_df = pd.DataFrame(new_list, columns=["en_page_title"])

# Display the DataFrame
new_df.head()

en_page_title
0    The Adventure of the Glowing Giant
1        The Hedgehog's Starry Surprise
2  The Magic Book and the Story Sprites
3          The Tale of the Rainbow Lion
4              The Night the Ocean Sang

In [ ]:
pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.2 MB/s eta 0:00:00


In [ ]:
from openai import OpenAI
import tiktoken
import random
import time

# List to store the generated summaries
en_summaries = []
token_counts = []

# Load the API key from the text file
with open('/content/api_key2.txt', 'r') as file:
    api_key = file.read().strip()

# Initialize the OpenAI client using the loaded API key
client = OpenAI(
    base_url="https://api.scaleway.ai/57bc9eaf-2692-4c43-bd14-7bf31594d232/v1",
    api_key=api_key
)

# Initialize tiktoken encoder for the "gpt-4o" model
enc = tiktoken.encoding_for_model("gpt-4o")
num_rows = len(new_df)
titles = new_df["en_page_title"].tolist()[:num_rows]

# Define a list of possible opening phrases to vary the beginning of the story
opening_phrases = [
    "Long ago in a distant land,",
    "There was a time when",
    "In a magical kingdom,",
    "In a faraway place,",
    "Not too long ago,",
    "Deep in the enchanted forest,",
    "High on a mountain top,",
    "In a tiny village,",
    "In a beautiful land of wonder,",
    "In the middle of a bustling town,"
]

# Loop over each title and generate summaries using the OpenAI API
backoff_time = 10  # initial backoff time in seconds
for i in range(num_rows):
    title = titles[i]

    # Select a random opening phrase for variety
    opening = random.choice(opening_phrases)

    # Create the messages for the ChatCompletion API
    messages = [
        {"role": "system", "content": "You are a helpful assistant who creates child-friendly, engaging versions of stories. Do not start the story with 'Once upon a time'. Use creative and different beginnings instead."},
        {"role": "user", "content": (
            f"Reformulate the story of '{title}' into a children's tale, telling it in a way that is engaging and easy for young children to understand. "
            f"Make sure the story feels like an enchanting bedtime tale, with a simple and friendly tone suitable for young children. Start with this: '{opening}'"
        )}
    ]

    # Calculate token count for the messages
    total_tokens = sum(len(enc.encode(message["content"])) for message in messages)
    print(f"Total tokens for row {i + 1}: {total_tokens}")

    # Generate the completion
    success = False
    while not success:
        try:
            response = client.chat.completions.create(
                model="llama-3.1-70b-instruct",
                messages=messages,
                max_tokens=512,
                temperature=0.7,
                top_p=0.7,
                presence_penalty=0,
                stream=True,
            )

            # Extract the generated response by iterating through chunks
            response_text = ""
            for chunk in response:
                if chunk.choices[0].delta.content is not None:
                    response_text += chunk.choices[0].delta.content

            #print(response_text)

            # Store the summary
            en_summary = response_text.strip()
            en_summaries.append(en_summary)

            # Reset backoff time if successful
            backoff_time = 5
            success = True

        except Exception as e:
            # Handle rate limit errors or other exceptions
            print(f"Error generating story for '{title}': {e}")
            print(f"Waiting for {backoff_time} seconds before retrying...")
            time.sleep(backoff_time)
            backoff_time = min(backoff_time * 2, 300)  # double the backoff time but limit to 5 minutes

    # Store the token count
    token_counts.append(total_tokens)

# Store the generated summaries in the DataFrame
new_df["en_concise_story"] = en_summaries

Total tokens for row 1: 103
In a faraway place, where the sun dipped into the horizon and painted the sky with hues of pink and orange, there lived a magical giant. This giant was unlike any other, for his body glowed softly in the dark, like a big, friendly lantern.

The giant's name was Glimmer, and he loved to roam the forest at night, his glowing body lighting the way for all the creatures who lived there. The fireflies would dance around him, the rabbits would hop alongside him, and the birds would sing sweet melodies to guide him on his way.

One evening, a little girl named Lily wandered into the forest, searching for a special flower to give to her mother. As she wandered deeper into the woods, the trees grew taller, and the path grew darker. But then, she saw a gentle glow in the distance.

As Lily approached the light, she saw Glimmer, the magical giant, standing tall and smiling at her. His glowing body made her feel safe and cozy, and she wasn't afraid at all. Glimmer spoke

KeyboardInterrupt: 

In [ ]:
# Check if we have fewer summaries than the number of rows in the DataFrame
if len(en_summaries) < len(new_df):
    # Add `None` for the rows that don't have summaries
    en_summaries.extend([None] * (len(new_df) - len(en_summaries)))

# Assign summaries to the DataFrame
new_df["en_concise_story"] = en_summaries

In [ ]:
new_df

en_page_title  \
0         The Adventure of the Glowing Giant   
1             The Hedgehog's Starry Surprise   
2       The Magic Book and the Story Sprites   
3               The Tale of the Rainbow Lion   
4                   The Night the Ocean Sang   
..                                       ...   
206       The Bear's Rainbow Slide Adventure   
207        The Tale of the Gigantic Mushroom   
208  The Panda's Whispering Stream Adventure   
209       The Giggles of the Crystal Fairies   
210           The Tale of the Moonlit Bridge   

                                      en_concise_story  
0    In a faraway place, where the sun dipped into ...  
1    Not too long ago, in a forest filled with tall...  
2    In a faraway place, where stars twinkled like ...  
3    There was a time when the savannah was painted...  
4    In a faraway place, where the sun dipped into ...  
..                                                 ...  
206                                               None  
207                                               None  
208                                               None  
209                                               None  
210                                               None  

[211 rows x 2 columns]

In [ ]:
from openai import OpenAI
import tiktoken
import random
import time

# Load the API key from the text file
with open('/content/api_key2.txt', 'r') as file:
    api_key = file.read().strip()

# Initialize the OpenAI client using the loaded API key
client = OpenAI(
    base_url="https://api.scaleway.ai/57bc9eaf-2692-4c43-bd14-7bf31594d232/v1",
    api_key=api_key
)

# Initialize tiktoken encoder for the "gpt-4o" model
enc = tiktoken.encoding_for_model("gpt-4o")
num_rows = len(new_df)
titles = new_df["en_page_title"].tolist()[:num_rows]

# Define a list of possible opening phrases to vary the beginning of the story
opening_phrases = [
    "Long ago in a distant land,",
    "There was a time when",
    "In a magical kingdom,",
    "In a faraway place,",
    "Not too long ago,",
    "Deep in the enchanted forest,",
    "High on a mountain top,",
    "In a tiny village,",
    "In a beautiful land of wonder,",
    "In the middle of a bustling town,"
]

# Define the starting point to continue generating summaries (from row 62)
start_index = 62

# List to store the generated summaries from the new point onward
en_summaries = list(new_df["en_concise_story"][:start_index])  # Copy existing summaries up to row 62
token_counts = []  # Will store token counts for the newly generated summaries

# Loop over each title starting from row 62 and generate summaries using the OpenAI API
backoff_time = 10  # initial backoff time in seconds
for i in range(start_index, num_rows):
    title = titles[i]

    # Select a random opening phrase for variety
    opening = random.choice(opening_phrases)

    # Create the messages for the ChatCompletion API
    messages = [
        {"role": "system", "content": "You are a helpful assistant who creates child-friendly, engaging versions of stories. Do not start the story with 'Once upon a time'. Use creative and different beginnings instead."},
        {"role": "user", "content": (
            f"Reformulate the story of '{title}' into a children's tale, telling it in a way that is engaging and easy for young children to understand. "
            f"Make sure the story feels like an enchanting bedtime tale, with a simple and friendly tone suitable for young children. Start with this: '{opening}'"
        )}
    ]

    # Calculate token count for the messages
    total_tokens = sum(len(enc.encode(message["content"])) for message in messages)
    print(f"Total tokens for row {i + 1}: {total_tokens}")

    # Generate the completion
    success = False
    while not success:
        try:
            response = client.chat.completions.create(
                model="llama-3.1-70b-instruct",
                messages=messages,
                max_tokens=512,
                temperature=0.7,
                top_p=0.7,
                presence_penalty=0,
                stream=True,
            )

            # Extract the generated response by iterating through chunks
            response_text = ""
            for chunk in response:
                if chunk.choices[0].delta.content is not None:
                    response_text += chunk.choices[0].delta.content

            # Store the summary
            en_summary = response_text.strip()
            en_summaries.append(en_summary)

            # Reset backoff time if successful
            backoff_time = 5
            success = True

        except Exception as e:
            # Handle rate limit errors or other exceptions
            print(f"Error generating story for '{title}': {e}")
            print(f"Waiting for {backoff_time} seconds before retrying...")
            time.sleep(backoff_time)
            backoff_time = min(backoff_time * 2, 300)  # double the backoff time but limit to 5 minutes

    # Store the token count
    token_counts.append(total_tokens)

# Store the generated summaries in the DataFrame
new_df["en_concise_story"] = en_summaries

Total tokens for row 63: 102
Total tokens for row 64: 101
Total tokens for row 65: 102
Total tokens for row 66: 105
Error generating story for 'The Giggles of the Glittering Goblin': Error code: 429 - {'message': 'You exceeded your current quota of tokens per minute', 'error': 'TOO MANY TOKENS', 'status': 429}
Waiting for 5 seconds before retrying...
Total tokens for row 67: 102
Total tokens for row 68: 103
Total tokens for row 69: 103
Error generating story for 'The Fairy's Firefly Lantern': Error code: 429 - {'error': 'TOO MANY TOKENS', 'status': 429, 'message': 'You exceeded your current quota of tokens per minute'}
Waiting for 5 seconds before retrying...
Error generating story for 'The Fairy's Firefly Lantern': Error code: 429 - {'error': 'TOO MANY TOKENS', 'message': 'You exceeded your current quota of tokens per minute', 'status': 429}
Waiting for 10 seconds before retrying...
Total tokens for row 70: 104
Total tokens for row 71: 104
Error generating story for 'The Panda's Secre

KeyboardInterrupt: 

In [ ]:
# Store the generated summaries in the DataFrame, only for the first 150 rows
new_df["en_concise_story"] = pd.Series(en_summaries[:150])

# Print the first 150 rows of the DataFrame
print(new_df.head(150))

                             en_page_title  \
0       The Adventure of the Glowing Giant   
1           The Hedgehog's Starry Surprise   
2     The Magic Book and the Story Sprites   
3             The Tale of the Rainbow Lion   
4                 The Night the Ocean Sang   
..                                     ...   
145           The Fox's Giggling Adventure   
146         The Panda's Glittering Garland   
147           The Tale of the Crystal Cave   
148  The Dancing Unicorn and the Star Tree   
149          The Day the River Glowed Pink   

                                      en_concise_story  
0    In a faraway place, where the sun dipped into ...  
1    Not too long ago, in a forest filled with tall...  
2    In a faraway place, where stars twinkled like ...  
3    There was a time when the savannah was painted...  
4    In a faraway place, where the sun dipped into ...  
..                                                 ...  
145  In a tiny village, where sunflowers bloomed

In [ ]:
new_df = new_df[:150]

In [ ]:
dff

fr_page_title  \
0    Les Aventures d'Alice au pays des merveilles   
1                                       Peter Pan   
2                                  Robin des Bois   
3                                           Mulan   
4                                      Pocahontas   
..                                            ...   
96                     L'Histoire sans fin (film)   
97                     La Fabuleuse Gilly Hopkins   
98                               Un chant de Noël   
99                  Le Secret de la Petite Sirène   
100                     Les Trois Brigands (film)   

                                           fr_page_url  \
0    https://fr.wikipedia.org/wiki/Alice_au_Pays_de...   
1              https://fr.wikipedia.org/wiki/Peter_Pan   
2         https://fr.wikipedia.org/wiki/Robin_des_Bois   
3                  https://fr.wikipedia.org/wiki/Mulan   
4             https://fr.wikipedia.org/wiki/Pocahontas   
..                                                 ...   
96   https://fr.wikipedia.org/wiki/L%27Histoire_san...   
97   https://fr.wikipedia.org/wiki/La_Fabuleuse_Gil...   
98      https://fr.wikipedia.org/wiki/Un_Chant_de_Noël   
99   https://fr.wikipedia.org/wiki/Le_Secret_de_la_...   
100  https://fr.wikipedia.org/wiki/Les_Trois_Brigan...   

                                    fr_article_extract  fr_word_count  \
0    « Alice au pays des merveilles » redirige ici....           2706   
1    Pour les articles homonymes, voir Peter Pan (h...           2996   
2    Pour les articles homonymes, voir Robin des Bo...           1830   
3    Série Classiques d'animation Disney Hercule(19...           2238   
4    Pour les articles homonymes, voir Pocahontas (...           2284   
..                                                 ...            ...   
96   Pour les articles homonymes, voir L'Histoire s...           1355   
97   Cet article est une ébauche concernant un film...            131   
98   A Christmas Carol Vous lisez un « article de q...          13449   
99   Pour les articles homonymes, voir La Petite Si...            856   
100  Pour les articles homonymes, voir Les Trois Br...            242   

                                     fr_internal_links  \
0    Accueil; Portails thématiques; Article au hasa...   
1    Accueil; Portails thématiques; Article au hasa...   
2    Accueil; Portails thématiques; Article au hasa...   
3    Accueil; Portails thématiques; Article au hasa...   
4    Accueil; Portails thématiques; Article au hasa...   
..                                                 ...   
96   Accueil; Portails thématiques; Article au hasa...   
97   Accueil; Portails thématiques; Article au hasa...   
98   Accueil; Portails thématiques; Article au hasa...   
99   Accueil; Portails thématiques; Article au hasa...   
100  Accueil; Portails thématiques; Article au hasa...   

                                         fr_categories  \
0    Catégories; Alice au pays des merveilles; Roma...   
1    Catégories; Peter Pan; Œuvre de littérature d'...   
2    Catégories; Robin des Bois; Archer de fiction;...   
3    Catégories; Film américain sorti en 1998; Film...   
4    Catégories; Dirigeant politique du XVIIe siècl...   
..                                                 ...   
96   Catégories; L'Histoire sans fin; Film réalisé ...   
97   Catégories; Film américain sorti en 2015; Comé...   
98   Catégories; Un chant de Noël; Noël dans la lit...   
99   Catégories; Film américain sorti en 2008; Film...   
100  Catégories; Comédie allemande; Film allemand s...   

                                             fr_images  \
0    /static/images/icons/wikipedia.png; /static/im...   
1    /static/images/icons/wikipedia.png; /static/im...   
2    /static/images/icons/wikipedia.png; /static/im...   
3    /static/images/icons/wikipedia.png; /static/im...   
4    /static/images/icons/wikipedia.png; /static/im...   
..                                                 ...   
96   /static/images/icons

In [ ]:
new_df

en_page_title  \
0       The Adventure of the Glowing Giant   
1           The Hedgehog's Starry Surprise   
2     The Magic Book and the Story Sprites   
3             The Tale of the Rainbow Lion   
4                 The Night the Ocean Sang   
..                                     ...   
145           The Fox's Giggling Adventure   
146         The Panda's Glittering Garland   
147           The Tale of the Crystal Cave   
148  The Dancing Unicorn and the Star Tree   
149          The Day the River Glowed Pink   

                                      en_concise_story  
0    In a faraway place, where the sun dipped into ...  
1    Not too long ago, in a forest filled with tall...  
2    In a faraway place, where stars twinkled like ...  
3    There was a time when the savannah was painted...  
4    In a faraway place, where the sun dipped into ...  
..                                                 ...  
145  In a tiny village, where sunflowers bloomed br...  
146  In the middle of a bustling town, where colorf...  
147  In a faraway place, where sparkling stars twin...  
148  There was a time when the moon shone bright in...  
149  In a faraway place, where the sun dipped into ...  

[150 rows x 2 columns]

In [ ]:
import pandas as pd

# Load the original df-finetune.csv
data_path = "/content/df-finetune.csv"
dff = pd.read_csv(data_path)

# Concatenate new_df and dff
combined_df = pd.concat([new_df, dff], ignore_index=True)

# Display the combined DataFrame
print(combined_df)

# Optionally, save the updated DataFrame to a new CSV file
combined_df.to_csv("/content/combined_df.csv", index=False)

                            en_page_title  \
0      The Adventure of the Glowing Giant   
1          The Hedgehog's Starry Surprise   
2    The Magic Book and the Story Sprites   
3            The Tale of the Rainbow Lion   
4                The Night the Ocean Sang   
..                                    ...   
246                 The Neverending Story   
247               The Great Gilly Hopkins   
248                     A Christmas Carol   
249                    The Little Mermaid   
250                     The Three Robbers   

                                      en_concise_story  \
0    In a faraway place, where the sun dipped into ...   
1    Not too long ago, in a forest filled with tall...   
2    In a faraway place, where stars twinkled like ...   
3    There was a time when the savannah was painted...   
4    In a faraway place, where the sun dipped into ...   
..                                                 ...   
246  Not too long ago, in a world much like ours, t..

In [27]:
import pandas as pd

data_path = "/content/combined_df.csv"
combined_df = pd.read_csv(data_path)

In [29]:
import pandas as pd

# Load combined DataFrame if needed
# combined_df = pd.read_csv("/content/combined_df_corrected.csv")

# Check the type of each column
print(combined_df.dtypes)

# Check if there are any NaN values in the columns
print("Number of NaN values in 'en_page_title':", combined_df["en_page_title"].isna().sum())
print("Number of NaN values in 'en_concise_story':", combined_df["en_concise_story"].isna().sum())

# Drop rows with NaN values in either of the columns
combined_df = combined_df.dropna(subset=["en_page_title", "en_concise_story"])

# Display the updated DataFrame to verify
#print(combined_df.head())
combined_df = combined_df.iloc[:, :2]
# Save the cleaned DataFrame to a new CSV file
combined_df.to_csv("/content/combined_df_cleaned.csv", index=False)

en_page_title       object
en_concise_story    object
dtype: object
Number of NaN values in 'en_page_title': 0
Number of NaN values in 'en_concise_story': 0


# IV - Finetuning (cf Ali's notebook)

In [ ]:
pip install transformers datasets peft bitsandbytes

In [ ]:
import pandas as pd
import random
from datasets import Dataset

# Load dataset from CSV
data_path = "/content/df-finetune.csv"
df = pd.read_csv(data_path)

# Select the columns to train with
df = df[["en_page_title", "en_concise_story"]]

# Rename columns for easier understanding
df = df.rename(columns={"en_page_title": "prompt", "en_concise_story": "completion"})

# Define variations for asking the story about each title
story_prompts = [
    "Tell me the story about {}",
    "Can you tell me the story of {}?",
    "I'd like to hear the story of {}.",
    "Please share the story of {} in a simple way.",
    "What is the story of {}?",
    "Could you narrate the story of {} for me?",
    "Can you tell a child-friendly version of the story of {}?",
    "Please tell me a simple story about {}.",
    "What happens in the story of {}?",
    "Could you tell me the tale of {} in an engaging way?"
]

# Update the "prompt" column with variations
new_prompts = []
for title in df["prompt"]:
    prompt_variation = random.choice(story_prompts).format(title)
    new_prompts.append(prompt_variation)

df["prompt"] = new_prompts

# Convert to Hugging Face dataset
dataset = Dataset.from_pandas(df)

# Display the updated DataFrame
df.head()


In [ ]:
!pip install --upgrade mistral_common
!pip install --upgrade vllm


In [ ]:
import pandas as pd
import random
from datasets import Dataset
from transformers import TrainingArguments, Trainer, AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model, TaskType
import torch

# Load the tokenizer
model_name = "EleutherAI/gpt-neo-2.7B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# Tokenize function
def tokenize_function(examples):
    return tokenizer(
        examples["prompt"],
        text_pair=examples["completion"],
        truncation=True,
        padding="max_length",
        max_length=512
    )

# Tokenize dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True)

In [ ]:
# Load the model
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)
model = model.to("cuda")

# Apply LoRA using PEFT library
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
)
model = get_peft_model(model, lora_config)

In [ ]:
# Split dataset into training and evaluation datasets
split_dataset = tokenized_dataset.train_test_split(test_size=0.2)
train_dataset = split_dataset['train']
eval_dataset = split_dataset['test']

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./finetuned_mistral",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    fp16=True,  # Use mixed precision for faster training
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)

In [ ]:
# Split dataset into training and evaluation datasets
split_dataset = tokenized_dataset.train_test_split(test_size=0.2)
train_dataset = split_dataset['train']
eval_dataset = split_dataset['test']

In [ ]:
# Start training
trainer.train()

# Save the fine-tuned model
trainer.save_model("./finetuned_mistral_model")

----

In [ ]:
pip install -U "huggingface_hub[cli]"

In [ ]:
!git config --global credential.helper store

In [ ]:
!huggingface-cli login

In [ ]:
!pip install bitsandbytes accelerate
!pip install transformers peft datasets

In [ ]:
from huggingface_hub import login

# Path to the file containing the token
token_file_path = "/content/hf_token.txt"

# Load the token from the file
with open(token_file_path, "r") as file:
    hf_token = file.read().strip()  # Read and remove any extra spaces or newline characters

# Use the loaded token to log in
login(token=hf_token)

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model, TaskType
from transformers import BitsAndBytesConfig

# Step 1: Load the Llama tokenizer and model from Hugging Face Hub
#model_name = "meta-llama/Meta-Llama-3-8B"
#model_name = "stas/tiny-random-llama-2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

#tokenizer = AutoTokenizer.from_pretrained(model_name, token=True)

# Using BitsAndBytesConfig for quantization
#bnb_config = BitsAndBytesConfig(load_in_8bit=True)

# Loading model with quantization config
"""
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    load_in_8bit=True,
    device_map="auto",
    use_auth_token=True
)
"""


In [ ]:

# Step 2: Apply LoRA to the model
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
)
model = get_peft_model(model, lora_config)


In [ ]:
# Add a padding token to the tokenizer
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Step 3: Tokenize the dataset
def tokenize_function(examples):
    tokens = tokenizer(
        examples["prompt"],
        text_pair=examples["completion"],
        truncation=True,
        padding="max_length",
        max_length=512
    )
    tokens["labels"] = tokens["input_ids"].copy()  # Use the input_ids as labels for causal language modeling
    return tokens

tokenized_dataset = dataset.map(tokenize_function, batched=True)

In [ ]:
training_args = TrainingArguments(
    output_dir="./finetuned_llama",
    evaluation_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,  # Accumulate gradients for 4 steps
    num_train_epochs=15,
    weight_decay=0.01,
    logging_steps=50,
    save_steps=500,
    save_total_limit=2,
    fp16=True,
    push_to_hub=False,
)

In [ ]:
pip install evaluate

In [ ]:
import evaluate
from transformers import DataCollatorForLanguageModeling, Trainer, TrainingArguments, AutoTokenizer, AutoModelForCausalLM
import torch

# Split the dataset into training and evaluation datasets (e.g., 80% train, 20% eval)
split_dataset = tokenized_dataset.train_test_split(test_size=0.2)

train_dataset = split_dataset['train']
eval_dataset = split_dataset['test']

# Step 5: Initialize Trainer and train the model

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # Set mlm to False since we are doing causal language modeling
)

# Load the accuracy metric from the evaluate library
accuracy_metric = evaluate.load("accuracy")

# Define a custom function to compute metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # Predictions are the argmax of logits
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    # Compute accuracy using the metric
    labels = torch.tensor(labels)
    # Since language modeling could have multiple predictions per batch, flatten all predictions and labels
    predictions = predictions.view(-1)
    labels = labels.view(-1)
    # Mask the padding
    mask = labels != -100
    predictions = predictions[mask]
    labels = labels[mask]
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    return {"accuracy": accuracy['accuracy']}

# Use this collator in the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics  # Add the compute metrics function here
)


In [ ]:
from transformers import DataCollatorForLanguageModeling


# Split the dataset into training and evaluation datasets (e.g., 80% train, 20% eval)
split_dataset = tokenized_dataset.train_test_split(test_size=0.2)

train_dataset = split_dataset['train']
eval_dataset = split_dataset['test']

# Step 5: Initialize Trainer and train the model

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # Set mlm to False since we are doing causal language modeling
)

# Use this collator in the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
)
# Start the training process

trainer.train()

In [ ]:
# Modify the training arguments to disable evaluation
training_args = TrainingArguments(
    output_dir="./finetuned_llama",
    evaluation_strategy="no",  # Disable evaluation during training
    learning_rate=2e-4,
    per_device_train_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=50,
    save_steps=500,
    save_total_limit=2,
    fp16=True,
    push_to_hub=False,
)

# Step 5: Initialize Trainer and train the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
)

In [ ]:

# Save the trained model
trainer.save_model("./finetuned_llama_lora")

------

In [ ]:
def preprocess_function(examples):
    # Concatenate prompt and completion for training
    return {"text": examples["prompt"] + " " + examples["completion"]}

# Apply preprocessing
dataset = dataset.map(preprocess_function)

In [ ]:
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [ ]:
# Adding a padding token to the tokenizer
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Update the model with the new padding token
model.resize_token_embeddings(len(tokenizer))

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

In [ ]:
split_dataset = tokenized_dataset.train_test_split(test_size=0.2)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

In [ ]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # Set mlm to False since we are doing causal language modeling
)

In [ ]:
pip install evaluate

In [ ]:
from evaluate import load

metric = load("accuracy")

In [ ]:
metric = load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.argmax(axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
# Training arguments setup
training_args = TrainingArguments(
    output_dir="./finetuned_llama",
    evaluation_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=50,
    save_steps=500,
    save_total_limit=2,
    fp16=True,
    push_to_hub=False,
)


In [ ]:
# Set up the Trainer with the compute_metrics function
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,  # Add this line
)


In [ ]:
trainer.train()

In [ ]:
trainer.save_model("./finetuned_llama_lora")